In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

PAGE_NUM = 0
ARTICLES = 20 # max 20
API_KEY = "pwm4EP6mJg6p3Ys2STFagWEkQMD0NqD3"

In [6]:
import pandas as pd

# Read both CSV files
df1 = pd.read_csv('fmp_1.csv')
df2 = pd.read_csv('fmp_2.csv')

# Stack them vertically
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save to new CSV file
combined_df.to_csv('fmp_full.csv', index=False)

In [2]:
def clean_html_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    # Заменяем <p> на переносы строк и удаляем пустые строки
    for p in soup.find_all('p'):
        p.append('\n\n')
    
    # Получаем текст и обрабатываем
    text = soup.get_text()
    
    # Удаляем лишние переносы и пробелы
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    clean_text = '\n'.join(lines)
    
    return clean_text.strip()


def parse_fmp_articles(data):
    articles = []
    for article in data["content"]:
        # Очистка HTML-тегов из контента
        clean_content = clean_html_text(article["content"])
        
        articles.append({
            "title": article["title"],
            "date": article["date"],
            "content": clean_content,  # Текст без HTML
            "tickers": article["tickers"],
            "image": article["image"],
            "link": article["link"],
            "author": article["author"],
            "source": article["site"]
        })
    return articles

In [4]:
articles = []

for page in range(52, 102):  # 20 articles per page → 50 pages = 1,000 articles
    url = f"https://financialmodelingprep.com/api/v3/fmp/articles?page={page}&size={ARTICLES}&apikey={API_KEY}"
    # https://financialmodelingprep.com/api/v3/fmp/articles?page=2&size=20&apikey=pwm4EP6mJg6p3Ys2STFagWEkQMD0NqD3
    response = requests.get(url).json()
    
    arts = parse_fmp_articles(response)

    articles.extend(arts)

df = pd.DataFrame(articles)[["content"]]
df.to_csv("fmp_2000.csv", index=False)

In [14]:
print(articles)

[{'title': 'Arteris, Inc. (AIP) Wins AI Engineering Innovation Award', 'date': '2025-06-27 18:00:07', 'content': 'Arteris, Inc. (NASDAQ:AIP) received the "AI Engineering Innovation Award" for its FlexGen technology, enhancing system-on-chip (SoC) development for AI applications.\nFlexGen technology significantly improves NoC design efficiency, with early adopters reporting a tenfold productivity increase and notable reductions in wire length and latency.\nDespite its technological advancements, Arteris faces financial challenges, including a negative P/E ratio of -12.18 and a high enterprise value to operating cash flow ratio of 236.35, indicating potential overvaluation.\nArteris, Inc. (Nasdaq: AIP) is a key player in the semiconductor industry, specializing in system IP for system-on-chip (SoC) development. The company recently received the "AI Engineering Innovation Award" at the 2025 AI Breakthrough Awards for its FlexGen technology. This innovation addresses the challenges of netw

In [ ]:
df = pd.DataFrame(articles)[["content"]]
df.to_csv("fmp_1000.csv", index=False)

In [29]:
print(len(df))
df["content"][0]

1000


'Arteris, Inc. (NASDAQ:AIP) received the "AI Engineering Innovation Award" for its FlexGen technology, enhancing system-on-chip (SoC) development for AI applications.\nFlexGen technology significantly improves NoC design efficiency, with early adopters reporting a tenfold productivity increase and notable reductions in wire length and latency.\nDespite its technological advancements, Arteris faces financial challenges, including a negative P/E ratio of -12.18 and a high enterprise value to operating cash flow ratio of 236.35, indicating potential overvaluation.\nArteris, Inc. (Nasdaq: AIP) is a key player in the semiconductor industry, specializing in system IP for system-on-chip (SoC) development. The company recently received the "AI Engineering Innovation Award" at the 2025 AI Breakthrough Awards for its FlexGen technology. This innovation addresses the challenges of network-on-chip (NoC) design, particularly for AI-centric semiconductors used in data centers and self-driving cars.\

In [1]:
import pandas as pd
import json

# Load the CSV file
csv_path = 'bbc.csv'
df_csv = pd.read_csv(csv_path)

# Load the JSONL file and convert to DataFrame
jsonl_path = 'fmp_full_sums.jsonl'
data = []
with open(jsonl_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
df_jsonl = pd.DataFrame(data)

# Make sure both DataFrames have the same column names
# If your CSV has different column names, adjust as needed
df_jsonl = df_jsonl.rename(columns={
    'text': 'text',
    'summary': 'summary'
})

# Concatenate the DataFrames
merged_df = pd.concat([df_csv, df_jsonl], ignore_index=True)

# Save the merged DataFrame back to CSV
merged_df.to_csv('merged_bbc_fmp3.csv', index=False)

print(f"Merged data saved to merged_bbc_fmp.csv")
print(f"Original CSV rows: {len(df_csv)}")
print(f"JSONL rows added: {len(df_jsonl)}")
print(f"Total merged rows: {len(merged_df)}")

Merged data saved to merged_bbc_fmp.csv
Original CSV rows: 2127
JSONL rows added: 2000
Total merged rows: 4127


In [33]:
from transformers import pipeline

pipe = pipeline("summarization", model="human-centered-summarization/financial-summarization-pegasus")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Converting from SentencePiece and Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast converters: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'Realm

In [ ]:
import pandas as pd
from transformers import pipeline
import json

# Load your CSV file
input_csv = "/content/fmp_1000.csv"
output_jsonl = "summaries.jsonl"

# Read the CSV file
df = pd.read_csv(input_csv)

# Process each row and save summaries
with open(output_jsonl, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        text = row['content']  # Assuming your column is named 'content'

        # Generate summary (adjust max_length/min_length as needed)
        summary = summary = pipe(
            text,  # Don't manually truncate - let the tokenizer handle it
            max_length=100,  # Reduced from 150
            min_length=30,
            do_sample=False,
            truncation=True  # Let tokenizer handle truncation properly
        )[0]['summary_text']

        # Create JSON object
        result = {
            "text": text,
            "summary": summary
        }

        # Write as JSON line
        f.write(json.dumps(result, ensure_ascii=False) + '\n')

        # Print progress every 10 items
        if index % 10 == 0:
            print(f"Processed {index + 1}/{len(df)} items")

print(f"Summarization complete! Results saved to {output_jsonl}")